### Script for reading data for Problem 4.1 (data_GripStrength.txt) in Applied Statistics 2021 exam:

This script reads the file __"data_GripStrength.txt"__ for Problem 4.1 in the exam of Applied Statistics 2021/22. This data files contains 84 entries in three columns: 

* **dominant hand** (0 - left hand, 1 - right hand),
* **grip strength** (N) of the dominant hand,
* **grip strength** (N) of the non-dominant hand.

The script was distributed along with the exam and the data file itself on the 20th of January 2022.

####  Author: 
- Troels Petersen (Niels Bohr Institute, petersen@nbi.dk)

####  Date:   
 - 19th of January 2022 (first version)

In [44]:
import random
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Modules to see files and folders in directories
from scipy import stats
from scipy.stats import binom, poisson, norm ,gamma           # Functions from SciPy Stats...
# import scipy.optimize as optimization
from scipy import optimize
from numpy.linalg import inv
from functools import partial
from fitter import Fitter
from astropy.modeling import models, fitting

In [45]:
import pandas as pd

In [46]:
sys.path.append('D:\my github\Siyi Applied Stats\Documents for JN\AppStat2021-main\External_Functions\\')
from ExternalFunctions import Chi2Regression, BinnedLH, UnbinnedLH
from ExternalFunctions import nice_string_output, add_text_to_ax    # Useful functions to print fit results on figure

plt.rcParams['font.size'] = 18     # Set some basic plotting parameters

In [47]:
dataframe = pd.read_csv('data_GripStrength.csv', header=0, index_col=None)
domhand, dom_grip, ndom_grip = dataframe.values.T
dataframe

,DominantHand,DominantGrip(N),NonDominantGrip(N)
0,1.0,293.24,241.50
1,1.0,547.39,552.01
2,0.0,578.85,495.98
3,1.0,342.30,342.18
4,1.0,337.79,347.14
...,...,...,...
79,1.0,335.67,305.39
80,1.0,362.91,336.60
81,1.0,659.39,623.32
82,1.0,292.84,254.46


Print the first 10 entries to check the content of the file:

In [48]:
for i in range(10):
    print(f"  dominant hand = {domhand[i]:4.2f}    dominant grip = {dom_grip[i]:6.2f} N    non-dominant grip = {ndom_grip[i]:6.2f} N")

  dominant hand = 1.00    dominant grip = 293.24 N    non-dominant grip = 241.50 N
  dominant hand = 1.00    dominant grip = 547.39 N    non-dominant grip = 552.01 N
  dominant hand = 0.00    dominant grip = 578.85 N    non-dominant grip = 495.98 N
  dominant hand = 1.00    dominant grip = 342.30 N    non-dominant grip = 342.18 N
  dominant hand = 1.00    dominant grip = 337.79 N    non-dominant grip = 347.14 N
  dominant hand = 0.00    dominant grip = 510.41 N    non-dominant grip = 493.16 N
  dominant hand = 0.00    dominant grip = 188.86 N    non-dominant grip = 212.97 N
  dominant hand = 0.00    dominant grip = 537.67 N    non-dominant grip = 497.41 N
  dominant hand = 0.00    dominant grip = 235.56 N    non-dominant grip = 228.56 N
  dominant hand = 1.00    dominant grip = 217.30 N    non-dominant grip = 218.23 N


In [ ]:
'''
# problem 4.1.1
x=int
x=0
for j in range(84):
    if domhand[j] == 1.00:
        x += 1
print(x/84)
a = np.sqrt(x/84*(1-x/84)/84)
print(a)
'''

In [ ]:
'''
# problem 4.1.2
mean_d=np.mean(dom_grip)
std_d=np.std(dom_grip)
mean_n=np.mean(ndom_grip)
std_n=np.std(ndom_grip)
print(mean_d,std_d**2)
print(mean_n,std_n**2)
print(dom_grip.size)
print(ndom_grip.size)
'''

In [ ]:
'''
# problem 4.1.3 f-test
s1_square = 20777.2  # variance1
n1=84  # number of sample1
s2_square = 18876.7  # variance2
n2=84  # number of sample2
if s1_square > s2_square:
    f = s1_square/s2_square
else: 
    f = s2_square/s1_square
df1=n1-1
df2=n2-1
print(f)
print(df1,df2)
'''

In [51]:

# problem 4.1.3 two-sample test with same variance,dof: n-2
mean1=418.48
s1= 137.4   # uncertainty1
n1=84
mean2=444.93
s2=144.1   # uncertainty2
n2=84
sp= np.sqrt(((n1-1)*s1**2+(n2-1)*s2**2)/(n1+n2-2))  # pooled estimate variance
t= (mean1-mean2)/np.sqrt(sp**2/n1 + sp**2/n2)
dof= n1+n2-2
print(t)
print(dof)
print(sp**2)


-1.2175279532498275
166
19821.784999999996


In [ ]:
'''
# problem 4.1.4
d1=np.zeros_like(domhand)
for j in range(84):
    d1[j]=dom_grip[j]-ndom_grip[j]
print(np.mean(d1))
print(np.std(d1))
'''

In [ ]:
# problem 4.1.5
righthand=np.ones_like(domhand)
lefthand=np.ones_like(domhand)
for j in range(84):
    if domhand[j] == 1.00:
        righthand[j] = dom_grip[j]
        lefthand[j] = ndom_grip[j]
    else:
        righthand[j] = ndom_grip[j]
        lefthand[j] = dom_grip[j]

print(np.mean(righthand))
print(np.std(righthand)**2)
print(np.mean(lefthand))
print(np.std(lefthand)**2)

In [52]:
# problem 4.1.5 f-test
s1_square = 20178.8  # variance1
n1=84  # number of sample1
s2_square = 19712.04  # variance2
n2=84  # number of sample2
if s1_square > s2_square:
    f = s1_square/s2_square
else: 
    f = s2_square/s1_square
df1=n1-1
df2=n2-1
print(f)
print(df1,df2)

1.023678929222952
83 83


In [53]:

# problem 4.1.5 two-sample test with same variance,dof: n-2
mean1=439.22
s1= 142.05   # uncertainty1
n1=84
mean2=424.2
s2=140.4   # uncertainty2
n2=84
sp= np.sqrt(((n1-1)*s1**2+(n2-1)*s2**2)/(n1+n2-2))  # pooled estimate variance
t= (mean1-mean2)/np.sqrt(sp**2/n1 + sp**2/n2)
dof= n1+n2-2
print(t)
print(dof)
print(sp**2)


0.6892481105350278
166
19945.18125
